In [107]:
from tqdm import tqdm_notebook as tqdm
from itertools import repeat
from numba import jit, int32
from numba.typed import List
import numpy as np

In [63]:
@jit(int32(int32, int32), nopython=True)
def ggt(a,b):
    a,b = abs(a), abs(b)
    while b!=0:
        a, b = b, a % b
    return a

@jit(int32(int32, int32), nopython=True)
def kgv(a,b):
    return a*b//ggt(a,b)

In [122]:
inp = List()
for x in list(map(int, list('12345678'))):
    inp.append(x)
inp

ListType[int64]([1, 2, 3, 4, 5, 6, 7, 8])

In [123]:
@jit(nopython=True)
def process(inp, i, pat, period = -1):
    new_pat = [0 for x in range(0)]
    for e in pat: new_pat += [e for l in range(i+1)]
    if period>0:
        rep = kgv(period, 4*i)
        #print(period, len(new_pat), rep)
        return (process(inp, i, pat) * (len(inp) // rep)) % 10
    #print(len(new_pat), new_pat)
    res = 0
    for j in range(len(inp)):
        res += inp[j] * new_pat[(j+1) % len(new_pat)]
        #print(i,j,inp[j], new_pat[(j+1) % len(new_pat)], res)
    res = res % 10 if res>=0 else -res % 10
    return res

In [127]:
@jit(nopython=True)
def step(inp, period=-1):
    pat = List()
    for x in [0, 1, 0, -1]: pat.append(x)
    res = List()
    for x in [process(inp, i, pat, period) for i in range(len(inp))]:
        res.append(x)
    return res

In [129]:
step(step(inp))

ListType[int64]([3, 4, 0, 4, 0, 4, 3, 8])

In [130]:
s='59731816011884092945351508129673371014862103878684944826017645844741545300230138932831133873839512146713127268759974246245502075014905070039532876129205215417851534077861438833829150700128859789264910166202535524896960863759734991379392200570075995540154404564759515739872348617947354357737896622983395480822393561314056840468397927687908512181180566958267371679145705350771757054349846320639601111983284494477902984330803048219450650034662420834263425046219982608792077128250835515865313986075722145069152768623913680721193045475863879571787112159970381407518157406924221437152946039000886837781446203456224983154446561285113664381711600293030463013'
inp = List()
for x in list(map(int, list(s))):
    inp.append(x)

In [131]:
%%timeit
x = inp.copy()
for n in range(100):
    #print(n,x)
    x = step(x)
x[:8]

2.73 s ± 72.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [138]:
s = '59731816011884092945351508129673371014862103878684944826017645844741545300230138932831133873839512146713127268759974246245502075014905070039532876129205215417851534077861438833829150700128859789264910166202535524896960863759734991379392200570075995540154404564759515739872348617947354357737896622983395480822393561314056840468397927687908512181180566958267371679145705350771757054349846320639601111983284494477902984330803048219450650034662420834263425046219982608792077128250835515865313986075722145069152768623913680721193045475863879571787112159970381407518157406924221437152946039000886837781446203456224983154446561285113664381711600293030463013'
period = len(s)
s = s * 10000
inp = List()
for x in list(map(int, list(s))):
    inp.append(x)

In [139]:
x = inp.copy()
for n in tqdm(range(100)):
    x = step(x, period)
off = int(''.join(x[:7]))
x[off:off+8]

ZeroDivisionError: integer division by zero

In [137]:
x

ListType[int64]([5, 9, 7, 3, 1, 8, 1, 6, 0, 1, 1, 8, 8, 4, 0, 9, 2, 9, 4, 5, 3, 5, 1, 5, 0, 8, 1, 2, 9, 6, 7, 3, 3, 7, 1, 0, 1, 4, 8, 6, 2, 1, 0, 3, 8, 7, 8, 6, 8, 4, 9, 4, 4, 8, 2, 6, 0, 1, 7, 6, 4, 5, 8, 4, 4, 7, 4, 1, 5, 4, 5, 3, 0, 0, 2, 3, 0, 1, 3, 8, 9, 3, 2, 8, 3, 1, 1, 3, 3, 8, 7, 3, 8, 3, 9, 5, 1, 2, 1, 4, 6, 7, 1, 3, 1, 2, 7, 2, 6, 8, 7, 5, 9, 9, 7, 4, 2, 4, 6, 2, 4, 5, 5, 0, 2, 0, 7, 5, 0, 1, 4, 9, 0, 5, 0, 7, 0, 0, 3, 9, 5, 3, 2, 8, 7, 6, 1, 2, 9, 2, 0, 5, 2, 1, 5, 4, 1, 7, 8, 5, 1, 5, 3, 4, 0, 7, 7, 8, 6, 1, 4, 3, 8, 8, 3, 3, 8, 2, 9, 1, 5, 0, 7, 0, 0, 1, 2, 8, 8, 5, 9, 7, 8, 9, 2, 6, 4, 9, 1, 0, 1, 6, 6, 2, 0, 2, 5, 3, 5, 5, 2, 4, 8, 9, 6, 9, 6, 0, 8, 6, 3, 7, 5, 9, 7, 3, 4, 9, 9, 1, 3, 7, 9, 3, 9, 2, 2, 0, 0, 5, 7, 0, 0, 7, 5, 9, 9, 5, 5, 4, 0, 1, 5, 4, 4, 0, 4, 5, 6, 4, 7, 5, 9, 5, 1, 5, 7, 3, 9, 8, 7, 2, 3, 4, 8, 6, 1, 7, 9, 4, 7, 3, 5, 4, 3, 5, 7, 7, 3, 7, 8, 9, 6, 6, 2, 2, 9, 8, 3, 3, 9, 5, 4, 8, 0, 8, 2, 2, 3, 9, 3, 5, 6, 1, 3, 1, 4, 0, 5, 6, 8, 4, 0, 4, 6, 8, 3, 9,